## Testing some python codes

In [1]:
import os
from os.path import join
import pandas as pd
import numpy as np
from behavior_selection import behavior_selection
from neural_matrix import neural_matrix
from neuro_behavior_options import neuro_behavior_options
from load_data import load_data

# figurePath = '/Users/paulmiddlebrooks/Projects/ridgeregress/docs/'
# bhvDataPath = '/Users/paulmiddlebrooks/Projects/neuro-behavior/data/processed_behavior/'
# nrnDataPath = '/Users/paulmiddlebrooks/Projects/neuro-behavior/data/raw_ephys/'
# saveDataPath = '/Users/paulmiddlebrooks/Projects/ridgeRegress/data/'
figurePath = 'E:/Projects/ridgeregress/docs/'
bhvDataPath = 'E:/Projects/neuro-behavior/data/processed_behavior/'
nrnDataPath = 'E:/Projects/neuro-behavior/data/raw_ephys/'
saveDataPath = 'E:/Projects/ridgeRegress/data/'


animal = 'ag25290'
sessionBhv = '112321_1'
sessionNrn = '112321'

if sessionBhv == '112321_1':
    sessionSave = '112321'



opts = neuro_behavior_options()

nrnDataPath = nrnDataPath + 'animal_' + animal + '/' + sessionNrn + '/recording1/'

bhvDataPath = join(bhvDataPath, 'animal_'+ animal)
bhvFileName = 'behavior_labels_' + animal + '_' + sessionBhv + '.csv'
opts.dataPath = bhvDataPath
opts.fileName = bhvFileName


In [2]:
figurePath = join(figurePath, animal, sessionSave, 'figures', f'start {opts.collectStart} for {opts.collectFor}')

if not os.path.exists(figurePath):
    os.makedirs(figurePath)


# data_full = pd.read_csv(join(opts.dataPath, opts.fileName))
dataBhv = load_data(opts, 'behavior')


e:\Projects\neuro-behavior\src\load_data.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_window.loc[:,'Time'] = data_window['Time'] - data_window['Time'].iloc[0]


In [3]:
codes = np.unique(dataBhv.bhvID)
behaviors = []  # behaviors: a Python list containing the behavior names
for iBhv in range(len(codes)):
    # first_idx = np.where(dataBhv['bhvID'] == codes[iBhv])[0][0]
    firstIdx = np.where(dataBhv['bhvID'] == codes[iBhv])[0][0]
    behaviors.append(dataBhv['bhvName'].iloc[firstIdx])

opts.behaviors = behaviors
opts.bhvCodes = codes
opts.validCodes = np.array(codes)[np.array(codes) != -1]

# Select valid behaviors
valiBhv = behavior_selection(dataBhv, opts)
opts.valid_bhv = valiBhv
# all_valid = np.sum(np.array(valiBhv), axis=1)  # A list of all the valid behavior indices

rmv_bhv = np.zeros(len(behaviors), dtype=int)
for i in range(len(behaviors)):
    if np.sum(valiBhv[:,i]) < 20:
        rmv_bhv[i] = 1

analyzeBhv = np.array(behaviors)[rmv_bhv == 0]
analyzeCodes = np.array(codes)[rmv_bhv == 0]


in_nest_sleeping_or_irrelevant code -1.0 is not a valid behavior for this analysis


Behavior 0.0: investigate_1
274: allPossible
274: andLongEnough
201: andNotRepeated
Percent valid: 73.4

Behavior 1.0: investigate_2
650: allPossible
650: andLongEnough
305: andNotRepeated
Percent valid: 46.9

Behavior 2.0: investigate_3
114: allPossible
114: andLongEnough
79: andNotRepeated
Percent valid: 69.3

Behavior 3.0: rear
35: allPossible
35: andLongEnough
30: andNotRepeated
Percent valid: 85.7

Behavior 4.0: dive_scrunch
112: allPossible
112: andLongEnough
104: andNotRepeated
Percent valid: 92.9

Behavior 5.0: paw_groom
74: allPossible
74: andLongEnough
45: andNotRepeated
Percent valid: 60.8

Behavior 6.0: face_groom_1
157: allPossible
157: andLongEnough
106: andNotRepeated
Percent valid: 67.5

Behavior 7.0: face_groom_2
60: allPossible
60: andLongEnough
38: andNotRepeated
Percent valid: 63.3

Behavior 8.0: head_groom
66: allPossible
66: andLongEnough
36: andNotRepeated
Percent valid: 54.5

Be

# Neural data

In [4]:
# Load data
opts.dataPath = nrnDataPath
data = load_data(opts, 'neuron')
# data.bhvDur = dataBhv.bhvDur
data["bhvDur"] = dataBhv.bhvDur
# spikeTimes = data.spikeTimes
# spikeClusters = data.spikeClusters


# Find neuron clusters (ids) in each brain region
allGood = (data['ci'].group.values == 'good') & (data['ci'].KSLabel.values == 'good')

goodM23 = allGood & (data['ci'].area == 'M23')
goodM56 = allGood & (data['ci'].area == 'M56')
goodCC = allGood & (data['ci'].area == 'CC')
goodDS = allGood & (data['ci'].area == 'DS')
goodVS = allGood & (data['ci'].area == 'VS')

opts.useNeurons = np.where(goodM23 | goodM56 | goodDS | goodVS)[0]

a = [np.sum(goodM23), np.sum(goodM56), np.sum(goodCC), np.sum(goodDS), np.sum(goodVS)]
np.sum(a)

# Make or load neural matrix
opts.useNeurons = np.where(goodM23 | goodM56 | goodDS | goodVS)[0]

dataMat, idLabels, areaLabels, removedNeurons = neural_matrix(data, opts)  # Change rrm_neural_matrix

idM23 = np.where(np.array(areaLabels) == 'M23')[0]
idM56 = np.where(np.array(areaLabels) == 'M56')[0]
idDS = np.where(np.array(areaLabels) == 'DS')[0]
idVS = np.where(np.array(areaLabels) == 'VS')[0]

print(f'{len(idM23)} M23\n{len(idM56)} M56\n{len(idDS)} DS\n{len(idVS)} VS')


Keeping 142 of 376 neurons
17 M23
28 M56
63 DS
34 VS


In [7]:
dataMat.shape

(18000, 142)

In [8]:
dataMat.shape


(18000, 230)